In [6]:
import os
import glob
import numpy as np
import miditoolkit
import tensorflow as tf

melody_folder = '/Users/ishitachawla/Documents/CSE253/Assignment_2/nottingham-dataset/MIDI/melody'
chord_folder  = '/Users/ishitachawla/Documents/CSE253/Assignment_2/nottingham-dataset/MIDI/chords'

In [8]:
all_melody_seqs = []
all_chord_seqs  = []

# We’ll sort the filenames so that melody and chord files line up by name.
melody_files = sorted(glob.glob(os.path.join(melody_folder, '*.mid')))
chord_files  = sorted(glob.glob(os.path.join(chord_folder,  '*.mid')))

print(f'Found {len(melody_files)} melody files and {len(chord_files)} chord files.')

Found 1034 melody files and 1021 chord files.


In [9]:
import os
import glob

# 1) Grab all .mid file paths
all_melody_paths = glob.glob(os.path.join(melody_folder, '*.mid'))
all_chord_paths  = glob.glob(os.path.join(chord_folder,  '*.mid'))

# 2) Build a dict: { base_name_without_ext → full_path }
melody_dict = {
    os.path.splitext(os.path.basename(m))[0]: m
    for m in all_melody_paths
}
chord_dict = {
    os.path.splitext(os.path.basename(c))[0]: c
    for c in all_chord_paths
}

# 3) Compute the intersection of base‐names
common_names = set(melody_dict.keys()).intersection(set(chord_dict.keys()))

# 4) Filter both lists to only keep those whose base‐name exists in BOTH
melody_files_filtered = [melody_dict[name] for name in sorted(common_names)]
chord_files_filtered  = [chord_dict[name]  for name in sorted(common_names)]


In [10]:
print(f'Found {len(melody_files_filtered)} melody files and {len(chord_files_filtered)} chord files after filtering.')

Found 1021 melody files and 1021 chord files after filtering.


In [13]:
all_melody_seqs = []
all_chord_seqs  = []

for m_path, c_path in zip(melody_files_filtered, chord_files_filtered):
    # (a) Load melody MIDI → instrument[0] → sorted notes by start time
    m_midi = miditoolkit.MidiFile(m_path)
    melody_instr = m_midi.instruments[0]
    melody_instr.notes.sort(key=lambda note: note.start)
    melody_pitches = [note.pitch for note in melody_instr.notes]
    all_melody_seqs.append(melody_pitches)

    # (b) Load chord MIDI → instrument[0] → sorted notes by start time
    c_midi = miditoolkit.MidiFile(c_path)
    chord_instr = c_midi.instruments[0]
    chord_instr.notes.sort(key=lambda note: note.start)
    chord_pitches = [note.pitch for note in chord_instr.notes]
    all_chord_seqs.append(chord_pitches)

print("Loaded pitch sequences from filtered MIDI files.")


Loaded pitch sequences from filtered MIDI files.


In [ ]:
unique_melody_pitches = sorted({p for seq in all_melody_seqs for p in seq})
unique_chord_pitches  = sorted({p for seq in all_chord_seqs  for p in seq})

melody_to_int = {pitch: idx for idx, pitch in enumerate(unique_melody_pitches)}
int_to_melody = {idx: pitch for pitch, idx in melody_to_int.items()}

chord_to_int  = {pitch: idx for idx, pitch in enumerate(unique_chord_pitches)}
int_to_chord  = {idx: pitch for pitch, idx in chord_to_int.items()}

print(f"Melody vocab size  M = {len(melody_to_int)}")
print(f"Chord   vocab size  K = {len(chord_to_int)}")

Melody vocab size  M = 33
Chord   vocab size  K = 22
Unique melody pitches: [55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88]
Unique chord pitches: [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57]


In [19]:
all_melody_ints = []
all_chord_ints  = []

for mel_seq, ch_seq in zip(all_melody_seqs, all_chord_seqs):
    mel_int_seq = [melody_to_int[p] for p in mel_seq]
    ch_int_seq  = [chord_to_int[p]  for p in ch_seq]
    all_melody_ints.append(mel_int_seq)
    all_chord_ints.append(ch_int_seq)

print("Converted all pitch sequences to integer sequences.")

print(f"Total melody sequences: {len(all_melody_ints)}")
print(f"Total chord sequences: {len(all_chord_ints)}")

Converted all pitch sequences to integer sequences.
Total melody sequences: 1021
Total chord sequences: 1021


In [29]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate
from tensorflow.keras.models import Model
class ConditionalMidiLSTM:
    def __init__(self,
                 melody_vocab_size,
                 chord_vocab_size,
                 sequence_length=25,
                 melody_embed_dim=64,
                 chord_embed_dim=64,
                 lstm_units=128,
                 dropout_rate=0.2):
        self.sequence_length   = sequence_length
        self.melody_vocab_size = melody_vocab_size
        self.chord_vocab_size  = chord_vocab_size
        self.melody_embed_dim  = melody_embed_dim
        self.chord_embed_dim   = chord_embed_dim
        self.lstm_units        = lstm_units
        self.dropout_rate      = dropout_rate
        self.model             = None

    def create_conditional_sequences(self, all_mel_lists, all_ch_lists):
        """
        all_mel_lists: list of lists of ints (each int in [0..M-1])
        all_ch_lists:  list of lists of ints (each int in [0..K-1]),
                       len(mel_list[i]) == len(ch_list[i]) for each i.

        Returns:
          X_melody: (N, L)
          X_chord:  (N, L)
          y_chord:  (N, K)  one-hot targets
        """
        sequences_m = []
        sequences_c = []
        targets_c   = []
        L = self.sequence_length
        step = L // 2  # half-overlap

        for m_seq, c_seq in zip(all_mel_lists, all_ch_lists):
            min_len = min(len(m_seq), len(c_seq))
            m_seq = m_seq[:min_len]
            c_seq = c_seq[:min_len]
            T = len(m_seq)
            for i in range(0, T - L, step):
                seq_m = m_seq[i : i + L]
                seq_c = c_seq[i : i + L]
                tgt   = c_seq[i + L]
                sequences_m.append(seq_m)
                sequences_c.append(seq_c)
                targets_c.append(tgt)

        X_melody = np.array(sequences_m, dtype=np.int32)
        X_chord  = np.array(sequences_c, dtype=np.int32)
        y_chord  = tf.keras.utils.to_categorical(
            targets_c, num_classes=self.chord_vocab_size
        )
        return X_melody, X_chord, y_chord

    def build_model(self):
        L = self.sequence_length
        M = self.melody_vocab_size
        K = self.chord_vocab_size
        em = self.melody_embed_dim
        ec = self.chord_embed_dim
        U  = self.lstm_units
        d  = self.dropout_rate

        # Melody branch
        melody_in  = Input(shape=(L,), name='melody_input')
        melody_emb = Embedding(input_dim=M, output_dim=em, input_length=L)(melody_in)

        # Chord branch (previous chords)
        chord_in   = Input(shape=(L,), name='chord_input')
        chord_emb  = Embedding(input_dim=K, output_dim=ec, input_length=L)(chord_in)

        # Concatenate along last axis → (L, em+ec)
        merged = Concatenate(axis=-1)([melody_emb, chord_emb])

        # Single LSTM over merged embeddings
        x = LSTM(U, dropout=d, recurrent_dropout=d)(merged)

        # Final Dense → next chord
        output = Dense(K, activation='softmax', name='chord_output')(x)

        model = Model(inputs=[melody_in, chord_in], outputs=output)
        model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
        self.model = model
        print(model.summary())
        return model

In [30]:
sequence_length = 25

cm = ConditionalMidiLSTM(
    melody_vocab_size=len(melody_to_int),
    chord_vocab_size =len(chord_to_int),
    sequence_length =sequence_length,
    melody_embed_dim=64,
    chord_embed_dim =64,
    lstm_units      =128,
    dropout_rate    =0.2
)
print(len(all_melody_ints), len(all_chord_ints))    
X_mel, X_ch, y_ch = cm.create_conditional_sequences(all_melody_ints, all_chord_ints)
print(f"X_melody shape: {X_mel.shape} X_chord shape: {X_ch.shape} y_chord shape: {y_ch.shape}")

1021 1021
X_melody shape: (10976, 25) X_chord shape: (10976, 25) y_chord shape: (10976, 22)


In [31]:
model = cm.build_model()

# Shuffle data
indices = np.arange(X_mel.shape[0])
np.random.shuffle(indices)
X_mel = X_mel[indices]
X_ch  = X_ch[indices]
y_ch  = y_ch[indices]

history = model.fit(
    x={'melody_input': X_mel, 'chord_input': X_ch},
    y=y_ch,
    epochs=30,
    batch_size=128,
    validation_split=0.2
)

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ melody_input        │ (None, 25)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ chord_input         │ (None, 25)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 25, 64)    │      2,112 │ melody_input[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 25, 64)    │      1,408 │ chord_input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 25, 128)   │          0 │ embedding[0][0],  │
│ (Concatenate)       │                   │            │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 128)       │    131,584 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ chord_output        │ (None, 22)        │      2,838 │ lstm[0][0]        │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 137,942 (538.84 KB)

 Trainable params: 137,942 (538.84 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.1785 - loss: 2.7206 - val_accuracy: 0.5792 - val_loss: 1.7688
Epoch 2/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.6362 - loss: 1.4639 - val_accuracy: 0.7969 - val_loss: 0.8250
Epoch 3/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.7912 - loss: 0.7914 - val_accuracy: 0.8078 - val_loss: 0.6617
Epoch 4/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.8054 - loss: 0.6542 - val_accuracy: 0.8192 - val_loss: 0.5962
Epoch 5/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.8154 - loss: 0.5903 - val_accuracy: 0.8324 - val_loss: 0.5376
Epoch 6/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.8220 - loss: 0.5546 - val_accuracy: 0.8342 - val_loss: 0.5115
Epoch 7/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.8314 - loss: 0.5175 - val_accuracy: 0.8379 - val_loss: 0.4949
Epoch 8/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.8365 - loss: 0.4813 - val_accuracy: 0.837

In [32]:
def generate_chords(model, seed_melody, seed_chords=None,
                    num_steps=100, temperature=1.0, int_to_chord=None):
    """
    model: trained Keras model expecting {'melody_input':(1,L), 'chord_input':(1,L)}
    seed_melody: array-like of ints, length = L
    seed_chords: array-like of ints, length = L (if None, will be zeros)
    """
    L = seed_melody.shape[0]
    if seed_chords is None:
        seed_chords = np.zeros(L, dtype=np.int32)

    generated = []
    melody_seq = seed_melody.tolist()
    chord_seq  = seed_chords.tolist()

    for _ in range(num_steps):
        m_in = np.array([melody_seq[-L:]])  # shape (1, L)
        c_in = np.array([chord_seq[-L:]])   # shape (1, L)

        preds = model.predict({'melody_input': m_in, 'chord_input': c_in}, verbose=0)[0]
        preds = np.log(preds + 1e-8) / temperature
        exp_preds = np.exp(preds)
        probs = exp_preds / np.sum(exp_preds)

        next_chord_id = np.random.choice(range(probs.shape[-1]), p=probs)
        generated.append(next_chord_id)
        chord_seq.append(next_chord_id)

    if int_to_chord is not None:
        return [int_to_chord[i] for i in generated]
    else:
        return generated


In [33]:
seed_melody = np.array(all_melody_ints[0][:sequence_length], dtype=np.int32)
seed_chords = np.zeros(sequence_length, dtype=np.int32)  # initial “no-chord” context

generated_chord_pitches = generate_chords(
    model,
    seed_melody,
    seed_chords=seed_chords,
    num_steps=100,
    temperature=1.0,
    int_to_chord=int_to_chord
)

print("Generated chord pitches (MIDI ints):", generated_chord_pitches)

Generated chord pitches (MIDI ints): [40, 43, 43, 36, 40, 43, 43, 38, 42, 45, 43, 47, 50, 36, 40, 43, 43, 47, 50, 45, 48, 52, 38, 42, 45, 48, 43, 47, 50, 36, 40, 43, 38, 42, 45, 48, 43, 47, 50, 43, 47, 50, 36, 40, 43, 38, 42, 45, 48, 43, 47, 50, 36, 40, 43, 38, 42, 45, 48, 43, 47, 50, 36, 40, 43, 43, 47, 50, 38, 42, 45, 43, 47, 50, 45, 48, 52, 38, 42, 45, 48, 43, 47, 50, 43, 47, 50, 45, 48, 52, 38, 42, 45, 48, 43, 47, 50, 43, 47, 50]


In [34]:
import miditoolkit

# 1) Map your generated chord‐IDs back to actual MIDI pitches
#    (Here, int_to_chord[i] is already the MIDI pitch for chord-ID i.)
generated_pitches = generated_chord_pitches  # e.g. [48, 55, 60, 55, …]

# 2) Create a new MidiFile (default ticks_per_beat = 480)
new_midi = miditoolkit.MidiFile()

# 3) Create an Instrument to hold your chord notes
#    program=0 → Acoustic Grand Piano (you can choose any program number)
chord_instr = miditoolkit.Instrument(program=0, is_drum=False, name='Generated_Chords')

# 4) Decide how long each chord lasts (in ticks). 
#    For example, 1 beat per chord → ticks_per_beat (usually 480)
tpb = new_midi.ticks_per_beat
ticks_per_chord = tpb

# 5) For each pitch in generated_pitches, create a Note at the appropriate time
for idx, pitch in enumerate(generated_pitches):
    start_tick = idx * ticks_per_chord
    end_tick   = start_tick + ticks_per_chord
    # velocity=100 is a reasonable default
    note = miditoolkit.Note(
        velocity=100,
        pitch=pitch,
        start=start_tick,
        end=end_tick
    )
    chord_instr.notes.append(note)

# 6) Sort notes by start time (good practice)
chord_instr.notes.sort(key=lambda n: n.start)

# 7) Add the Instrument to the MidiFile
new_midi.instruments.append(chord_instr)

# 8) Write the result to disk
output_path = 'generated_harmonization.mid'
new_midi.dump(output_path)

print(f"Written MIDI file → {output_path}")

Written MIDI file → generated_harmonization.mid
